In [1]:
import pandas as pd
import numpy as np

Load the dataset that was prepared previously

In [2]:
data = pd.read_csv('newsgroup_data')

In [3]:
data.head()

,id,message,class
0,0,Archive-name: atheism/resources Alt-atheism-ar...,0
1,1,Archive-name: atheism/introduction Alt-atheism...,0
2,2,In article <65974@mimsy.umd.edu> mangoe@cs.umd...,0
3,3,dmn@kepler.unh.edu (...until kings become phil...,0
4,4,In article <N4HY.93Apr5120934@harder.ccr-p.ida...,0


Set X and y values

In [4]:
X = data[['id', 'message']]
y = data['class']

Do a test/train split. Usualy one would do 25/75 split, but this can be changed

In [130]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [131]:
X_train = X_train[:7000]
y_train = y_train[:7000]

C:\Users\Marija\AppData\Local\Temp\ipykernel_5976\245487900.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_train = y_train[:7000]


In [132]:
X_test = X_test[:3000]
y_test = y_test[:3000]

C:\Users\Marija\AppData\Local\Temp\ipykernel_5976\2082664043.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_test = y_test[:3000]


Creating a matrix with messages as rows and words as columns (every word that appears in each message)

In [133]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train.message.values.astype('U'))
X_train_counts.shape

(7000, 80866)

Using the TF-IDF algorithm to cut the noise out of the word vectors.

In [134]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(7000, 80866)

Training the classifier

In [135]:
from sklearn.naive_bayes import MultinomialNB
NBModel = MultinomialNB().fit(X_train_tfidf, y_train)

Checking the model on test data

In [136]:
X_test_counts = count_vect.transform(X_test.message.values.astype('U'))
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
NB_predicted = NBModel.predict(X_test_tfidf)

In [137]:
predicted = pd.DataFrame(NB_predicted, columns =['Predicted'])
predicted.head()

,Predicted
0,14
1,15
2,7
3,3
4,8


In [138]:
comparison = pd.concat([y_test.reset_index()['class'],predicted], axis=1)

In [139]:
comparison['CorrectBool'] = np.where(comparison['class']==comparison['Predicted'], 1, 0)

In [140]:
comparison.shape[0]

3000

In [141]:
results = comparison.groupby('CorrectBool')['class'].count()

In [142]:
results/comparison.shape[0]

CorrectBool
0    0.258333
1    0.741667
Name: class, dtype: float64

The results are correct 74.2% of the time